# Installing full profile Tanzu Application Platform

In [ ]:
cd $HOME/projects/tap-bounty/install/scripts

In [ ]:
chmod 755 setUp.sh

In [ ]:
./setUp.sh 

In [ ]:
cd $HOME/projects/tap-bounty

Update tap-values.yaml to reflect your own environment

In [ ]:
tanzu package install tap -p tap.tanzu.vmware.com -v 1.0.0 --values-file tap-values.yml -n tap-install

In [ ]:
tanzu package installed get tap -n tap-install

In [ ]:
tanzu package installed list -A

## Configure Ingress and Routing

In [ ]:
kubectl get svc envoy -n tanzu-system-ingress

In [ ]:
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction start --zone="tanzu4u"
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction add 35.226.28.109 --name="tap-gui.tap.tanzu4u.net." --ttl="300" --type="A" --zone="tanzu4u"
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction execute --zone="tanzu4u"

In [ ]:
cd tap-ingress

In [ ]:
./configure-ingress.sh values-ingress.yaml

In [ ]:
nslookup tap-gui.tap.tanzu4u.net

In [ ]:
open -a "Google Chrome" https://tap-gui.tap.tanzu4u.net

## Deploy a sample Java application

In [ ]:
tanzu apps workload create tanzu-java-web-app \
--git-repo https://github.com/sreeramsunkara/tanzu-java-web-app \
--git-branch main \
--type web \
--label app.kubernetes.io/part-of=tanzu-java-web-app \
--namespace dev \
--yes

In [ ]:
tanzu apps workload get tanzu-java-web-app -n dev

In [ ]:
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction start --zone="tanzu4u"
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction add 35.226.28.109 --name="*.apps.tap.tanzu4u.net." --ttl="300" --type="A" --zone="tanzu4u"
gcloud beta dns --project=fluted-lambda-274409 record-sets transaction execute --zone="tanzu4u"